# Assignment 7: Predictive Modeling of Housing Prices in Philadelphia
**Due date: Wednesday, 12/7 by the end of the day**

## Part 1: Final Proposal - Mia Cherayil, Kendra Hills, Myron Bañez

The proposal can be found in the Github repository for this assignment under the file name 
"550 Final Proposal - Mia, Kendra, Myron.pdf"

## Part 2: Modeling Philadelphia's Housing Prices and Algorithmic Fairness


In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import hvplot.pandas
import carto2gpd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV


np.random.seed(42)
pd.options.display.max_columns = 999
%matplotlib inline

### 2.1 Load data from the Office of Property Assessment

Use `carto2gpd` to load data for **single-family** properties in Philadelphia that had their **last sale during 2021**.

Sources: 
- [OpenDataPhilly](https://www.opendataphilly.org/dataset/opa-property-assessments)
- [Metadata](http://metadata.phila.gov/#home/datasetdetails/5543865f20583086178c4ee5/representationdetails/55d624fdad35c7e854cb21a4/)

In [ ]:
carto_url = "https://phl.carto.com/api/v2/sql"

# The table name
table_name = "opa_properties_public"

# Only pull 2021 sales for single family residential properties
where = "sale_date >= '2021-01-01' and sale_date <= '2021-12-31'"
where = where + " and category_code_description IN ('SINGLE FAMILY', 'Single Family')"

# Run the query
salesRaw = carto2gpd.get(carto_url, table_name, where=where)
salesRaw

In [ ]:
# The feature columns being used
cols = [
    "sale_price",
    "total_livable_area",
    "total_area",
    "garage_spaces",
    "fireplaces",
    "number_of_bathrooms",
    "number_of_bedrooms",
    "number_stories",
    "exterior_condition",
    "zip_code",
    "geometry"
]

# Triming to these columns and removing NaNs
sales = salesRaw[cols].dropna()

# Trimming zip code to first five digits
sales['zip_code'] = sales['zip_code'].astype(str).str.slice(0, 5)

### 2.2 Load data for census tracts and neighborhoods

Load various Philadelphia-based regions we will use in our analysis.

- Census tracts can be downloaded from: https://opendata.arcgis.com/datasets/8bc0786524a4486bb3cf0f9862ad0fbf_0.geojson
- Neighborhoods can be downloaded from:
https://raw.githubusercontent.com/azavea/geo-data/master/Neighborhoods_Philadelphia/Neighborhoods_Philadelphia.geojson


In [ ]:
tracts = gpd.read_file("Census_Tracts_2010.geojson")
tracts

### 2.3 Spatially join the sales data and neighborhoods/census tracts.

Perform a spatial join, such that each sale has an associated neighborhood and census tract.

**Note:** after performing the first spatial join, you will need to use the `drop()` function to remove the `index_right` column; otherwise an error will be raised on the second spatial join about duplicate columns.

In [ ]:
sales = sales.to_crs(epsg=3857)
tracts = tracts.to_crs(epsg=3857)
phl_sales = gpd.sjoin(sales, tracts, predicate='within', how='inner').drop(columns=['index_right'])

phl_sales

### 2.4 Train a Random Forest on the sales data

You should follow the steps outlined in lecture to preprocess and train your model. 

**Extra credit: the students with the top 3 scores on the test set will receive extra credit (first place +3, second place +2, third place +1)**

**Requirements**
- Trim the sales data to those sales with prices between \\$3,000 and \\$1 million DONE
- Set up a pipeline that includes both numerical columns and categorical columns DONE
- Include one-hot encoded variables for the neighborhood of the sale DONE
- Use a 70/30% training/test split DONE
- Use GridSearchCV to perform a $k$-fold cross validation that optimize *at least 2* hyperparameters of the RandomForestRegressor
- After fitting your model and finding the optimal hyperparameters, you should evaluate the score ($R^2$) on the test set (the original 30% sample withheld)

**Notes**

- You are welcome to include additional features or perform any feature engineering that you want to try to improve the test accuracy
- You can also experiment with the prediction variable, e.g., try predicting sale price per sq ft. (or its log)

In [ ]:
# Trim Data
valid = (phl_sales['sale_price'] > 3000) & (phl_sales['sale_price'] < 1e6)
phl_sales = phl_sales.loc[valid]

phl_sales

In [ ]:
# Pipeline + One-Hot Encoder

# Numerical Columns
num_cols = [
    "total_livable_area",
    "total_area",
    "garage_spaces",
    "fireplaces",
    "number_of_bathrooms",
    "number_of_bedrooms",
    "number_stories",
] 

# Categorical Columns
cat_cols = ["exterior_condition", "zip_code"]

transformer = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ]
)

pipe = make_pipeline(
    transformer, RandomForestRegressor(n_estimators=10, 
                                       random_state=42)
)

In [ ]:
# Split the data 70/30
train_set, test_set = train_test_split(phl_sales, test_size=0.3, random_state=42)

y_train = np.log(train_set["sale_price"])
y_test = np.log(test_set["sale_price"])

# Fit
pipe.fit(train_set, y_train);

# Score
pipe.score(test_set, y_test)

In [ ]:
# GridSearchCV
model_step = "randomforestregressor"
param_grid = {
    f"{model_step}__n_estimators": [5, 10, 15, 20, 30],
    f"{model_step}__max_depth": [None, 2, 5, 7, 9, 13],
}

param_grid

# Create the grid and use 3-fold CV
grid = GridSearchCV(pipe, param_grid, cv=3, verbose=1)

# Run the search
grid.fit(train_set, y_train)

In [ ]:
# Best Estimator
grid.best_estimator_

# Best Parameter
grid.best_params_

In [ ]:
best_random = grid.best_estimator_
grid.score(test_set, y_test)

### 2.5 Calculate the percent error of your model predictions for each sale in the test set

Fit your best model and use it to make predictions on the test set.

**Note:** this should be the percent error in terms of **sale price** or **sale price per sq ft**. You'll need to convert if you predicted the log!

In [ ]:
data = phl_sales.loc[test_set.index]
data

In [ ]:
# Prediction
log_prediction = grid.best_estimator_.predict(test_set)

# Conversion from Log
data['prediction'] = np.exp(log_prediction)

# Percent Error
data['pct_error'] = ((data["sale_price"]/data["prediction"])/data["sale_price"]) * 100

### 2.6 Make a data frame with percent errors and census tract info for each sale in the test set

Create a data frame that has the property geometries, census tract data, and percent errors for all of the sales in the test set.

**Notes**

- When using the "train_test_split()" function, the index of the test data frame includes the labels from the original sales data frame
- You can use this index to slice out the test data from the original sales data frame, which should include the census tract info and geometries
- Add a new column to this data frame holding the percent error data



In [ ]:
# I believe that all the information needed for this section was already included in my dataframe.
data

### 2.8 Plot a map of the median percent error by census tract 

- You'll want to group your data frame of test sales by the `GEOID10` column and take the median of you percent error column
- Merge the census tract geometries back in and use geopandas to plot.

In [ ]:
census_tracts = data.groupby('GEOID10')

# Median Percent Error Column 
mpe = census_tracts['pct_error'].median()

# Merge Census Tracts
mpe = tracts.merge(mpe, on='GEOID10')

# Plot
fig, ax = plt.subplots(figsize=(10,10))
mpe.plot(ax=ax, 
                 column='pct_error',
                 cmap = 'inferno',
                 legend=True)
ax.set_title("Median Percent Error by Philly Census Tracts", fontsize=16)
ax.set_axis_off()

### 2.9 Compare the percent errors in Qualifying Census Tracts and other tracts 

[Qualifying Census Tracts](https://www.huduser.gov/portal/datasets/qct.html) are a poverty designation that HUD uses to allocate housing tax credits

- I've included a list of the census tract names that qualify in Philadelphia
- Add a new column to your dataframe of test set sales that is True/False depending on if the tract is a QCT
- Then, group by this new column and calculate the median percent error

**You should find that the algorithm's accuracy is significantly worse in these low-income, qualifying census tracts**

In [ ]:
qct = ['5',
 '20',
 '22',
 '28.01',
 '30.01',
 '30.02',
 '31',
 '32',
 '33',
 '36',
 '37.01',
 '37.02',
 '39.01',
 '41.01',
 '41.02',
 '56',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '69',
 '70',
 '71.01',
 '71.02',
 '72',
 '73',
 '74',
 '77',
 '78',
 '80',
 '81.01',
 '81.02',
 '82',
 '83.01',
 '83.02',
 '84',
 '85',
 '86.01',
 '86.02',
 '87.01',
 '87.02',
 '88.01',
 '88.02',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '98.01',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '119',
 '121',
 '122.01',
 '122.03',
 '131',
 '132',
 '137',
 '138',
 '139',
 '140',
 '141',
 '144',
 '145',
 '146',
 '147',
 '148',
 '149',
 '151.01',
 '151.02',
 '152',
 '153',
 '156',
 '157',
 '161',
 '162',
 '163',
 '164',
 '165',
 '167.01',
 '167.02',
 '168',
 '169.01',
 '169.02',
 '170',
 '171',
 '172.01',
 '172.02',
 '173',
 '174',
 '175',
 '176.01',
 '176.02',
 '177.01',
 '177.02',
 '178',
 '179',
 '180.02',
 '188',
 '190',
 '191',
 '192',
 '195.01',
 '195.02',
 '197',
 '198',
 '199',
 '200',
 '201.01',
 '201.02',
 '202',
 '203',
 '204',
 '205',
 '206',
 '208',
 '239',
 '240',
 '241',
 '242',
 '243',
 '244',
 '245',
 '246',
 '247',
 '249',
 '252',
 '253',
 '265',
 '267',
 '268',
 '271',
 '274.01',
 '274.02',
 '275',
 '276',
 '277',
 '278',
 '279.01',
 '279.02',
 '280',
 '281',
 '282',
 '283',
 '284',
 '285',
 '286',
 '287',
 '288',
 '289.01',
 '289.02',
 '290',
 '291',
 '293',
 '294',
 '298',
 '299',
 '300',
 '301',
 '302',
 '305.01',
 '305.02',
 '309',
 '311.01',
 '312',
 '313',
 '314.01',
 '314.02',
 '316',
 '318',
 '319',
 '321',
 '325',
 '329',
 '330',
 '337.01',
 '345.01',
 '357.01',
 '376',
 '377',
 '380',
 '381',
 '382',
 '383',
 '389',
 '390']

In [ ]:
# True/False Column of QCT
median['QCT'] = np.where(median['NAME10'].isin(qct), True, False)

# Group By and Calculation
median_byqct = median.groupby('QCT')
medianqct = median_byqct['pct_error'].median()

medianqct